## Data Merging and Cleaning

This file assembles two "master" dataframes for experimentation. The first dataframe contains every crime throughout each day, with associated daily weather information. The second dataframe contains the number of arrests each day with associated mean weather information.

In [ ]:
import pandas as pd

### Arrest Data

In [ ]:
nypd_historic = pd.read_csv('./input/NYPD_Arrests_Data__Historic_.csv')
nypd_historic.head()

In [ ]:
nypd_historic.describe()

In [ ]:
nypd_current = pd.read_csv('./input/NYPD_Arrest_Data__Year_to_Date_.csv')
nypd_current.head()

In [ ]:
nypd_current.describe()

In [ ]:
len(nypd_current['ARREST_DATE'].unique()) + len(nypd_historic['ARREST_DATE'].unique())

In [ ]:
len(nypd_current['ARREST_DATE']) + len(nypd_historic['ARREST_DATE'])

In [ ]:
nypd = nypd_current.append(nypd_historic)
nypd.head()

In [ ]:
len(nypd)

In [ ]:
def format_date_nypd(date):
    split = date.split('/')
    return split[2] + split[1] + split[0]

In [ ]:
nypd['date'] = nypd['ARREST_DATE'].apply(format_date_nypd)
nypd_sum = nypd.groupby(['date']).sum()
nypd_sum.head()

In [ ]:
len(nypd_sum)

### Weather Data
#### Hourly weather

In [ ]:
weath = pd.read_csv('./input/daily_new_york_data.csv')
weath = weath[['dt', 'temp', 'feels_like', 'temp_min', 'temp_max', 
               'humidity', 'wind_speed', 'wind_deg', 'rain_1h', 
               'rain_3h', 'snow_1h', 'snow_3h', 'clouds_all']]
weath.head()

In [ ]:
len(weath)

In [ ]:
# weath.dtypes

In [ ]:
from datetime import datetime
from pytz import timezone # for timezone awareness

def format_date_weath(dt):
    localtz = timezone('America/New_York')
    dt_unaware = datetime.fromtimestamp(dt)
    dt_aware = localtz.localize(dt_unaware, is_dst=None)
    return datetime.utcfromtimestamp(dt_aware).strftime('%Y%m%d')

In [ ]:
weath['date'] = weath['dt'].apply(lambda dt: datetime.utcfromtimestamp(dt).strftime('%Y%m%d'))
weath.head()

In [ ]:
weath_mean = weath
weath_mean[['rain_1h', 'rain_3h', 
            'snow_1h', 'snow_3h']] = weath_mean[['rain_1h', 'rain_3h', 
                                                'snow_1h', 'snow_3h']].fillna(value=0)

weath_mean = weath.groupby(['date']).agg({'temp':'mean', 'feels_like':'mean', 
                                        'temp_min': 'min', 'temp_max': 'max',
                                        'humidity': 'mean', 'wind_speed': 'mean',
                                        'wind_deg': 'mean', 'rain_1h': 'mean',
                                        'rain_3h': 'mean', 'snow_1h': 'mean',
                                        'snow_3h': 'mean', 'clouds_all': 'mean'})

weath_mean['date'] = weath_mean.index
weath_mean.head()

In [ ]:
weath_mean.dtypes

In [ ]:
len(weath_mean)

### Daily crimes: each crime with weather data

This dataframe contains more detail, including each arrest's description and location in the city, by borough. This dataset may or may not be used, depending if we have time to do extra fancy visualizations. Otherwise, the next dataframe showing daily summaries are what we'll focus on first.

In [ ]:
daily_nypd = nypd.set_index(['date'])
daily_crimes = pd.merge(left=daily_nypd, right=weath_mean, how='left',
                        left_index=True, right_index=True)
daily_crimes = daily_crimes[['PD_DESC', 'OFNS_DESC', 'ARREST_BORO', 'temp', 
                             'feels_like', 'temp_min', 'temp_max', 'humidity', 
                             'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 
                             'snow_1h', 'snow_3h']]
daily_crimes.rename(columns={'PD_DESC': 'pd_desc', 'OFNS_DESC': 'ofns_desc', 
                             'ARREST_BORO': 'arrest_boro'}, inplace=True)
daily_crimes.dropna(inplace=True)
daily_crimes.head()
# daily_crimes[760:810] # For checking that weather is matched with the correct date.

In [ ]:
len(daily_nypd)

In [ ]:
# daily_crimes.dtypes

In [ ]:
len(daily_crimes)

In [ ]:
def get_borough(b):
    if b == 'B':
        return 'The Bronx'
    elif b == 'K':
        return 'Brooklyn'
    elif b == 'M':
        return 'Manhattan'
    else:
        return 'Queens'

In [ ]:
daily_crimes['pd_desc'] = daily_crimes['pd_desc'].str.capitalize()
daily_crimes['ofns_desc'] = daily_crimes['ofns_desc'].str.capitalize()
daily_crimes['arrest_boro'] = daily_crimes['arrest_boro'].apply(get_borough)
daily_crimes.head()

In [ ]:
len(daily_crimes)

### Write to file

In [ ]:
daily_crimes.to_csv('./output/daily_crimes.csv')

### Daily means: day's arrest count and weather means

This dataframe will likely be our primary dataset, since it's giving us a day-by-day arrest count with the mean weather conditions for that day.

In [ ]:
daily_mean = daily_crimes.groupby(['date']).count()
daily_mean.drop(columns=['ofns_desc', 'arrest_boro'], inplace=True)
daily_mean.rename(columns={'pd_desc': 'num_arrests'}, inplace=True)
daily_mean = daily_mean[['num_arrests']]
daily_mean = pd.merge(left=daily_mean, right=weath_mean, 
                      how='left', left_index=True, right_index=True)
daily_mean.head()

In [ ]:
daily_mean['num_arrests'].sum()

In [ ]:
# daily_mean.dtypes

In [ ]:
len(daily_mean)

### Write to file

In [ ]:
daily_mean.to_csv('./output/daily_mean.csv')